In [2]:
from dotenv import load_dotenv
import os

load_dotenv()

True

In [3]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()

In [4]:
with open("sample_text.md") as f:
    sample_text = f.read()

with open("system_prompt.txt") as f:
    system_prompt = f.read()

In [5]:
from langchain.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
)
prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("system", "The sample text is the following. You should use that style to write a new text based on the user input. Sample text: " + sample_text),
    MessagesPlaceholder(variable_name="chat_history", optional=True),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

In [6]:
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpfull writter assistant. You wil be given an example article, from which you will have to replicate the syle. You will have a set of tools to write and modify a new article considering the same style and content. Always start by reading the state of the current article, it might be empty or have some paragraphs already written. Follow the instructions given by the user and modify the article accordingly. ')),
 SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='The sample text is the following. You should use that style to write a new text based on the user input. Sample text: # An AI Programming Itself?\nAutonomous Agents, and some incredible applications\n\nAfter GPT-4 was released, a very interesting capability was discovered: the model was able to recognise its own mistakes. If you asked ChatGPT to create a poem that rhymes in every verse it might have 70% of th

In [7]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

In [8]:
# Import things that are needed generically
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool, StructuredTool, tool


In [9]:
class Article:
    def __init__(self, path):
        self.path = path

    def __str__(self):
        with open(self.path) as f:
            return f.read()
    
    def add_paragraph(self, paragraph):
        with open(self.path, "a") as f:
            f.write(paragraph + "\n\n")

    def modify_paragraph(self, index, paragraph):
        with open(self.path) as f:
            paragraphs = f.readlines()
        paragraphs[index] = paragraph + "\n\n"
        with open(self.path, "w") as f:
            f.write("".join(paragraphs))

    def remove_paragraph(self, index):
        with open(self.path) as f:
            paragraphs = f.readlines()
        del paragraphs[index]
        with open(self.path, "w") as f:
            f.write("".join(paragraphs))

    def get_paragraph(self, index):
        with open(self.path) as f:
            paragraphs = f.readlines()
        return paragraphs[index]
    
    def get_all_paragraphs(self):
        """Returns a json object with all paragraphs and their indexes"""
        with open(self.path) as f:
            paragraphs = f.readlines()
        return {i: p for i, p in enumerate(paragraphs)}

In [10]:
current_article = Article("article.txt")

@tool
def get_all_paragraphs() -> str:
    """Returns a json object with all paragraphs and their indexes"""
    return current_article.get_all_paragraphs()

@tool
def add_paragraph(new_paragraph: str) -> str:
    """Adds a paragraph to the article"""
    current_article.add_paragraph(new_paragraph)
    return "Paragraph added"

@tool
def modify_paragraph(index: int, modified_paragraph: str) -> str:
    """Modifies a paragraph in the article"""
    current_article.modify_paragraph(index, modified_paragraph)
    return "Paragraph modified"

@tool
def remove_paragraph(index: int) -> str:
    """Removes a paragraph from the article"""
    current_article.remove_paragraph(index)
    return "Paragraph removed"

tools = [get_all_paragraphs, add_paragraph, modify_paragraph, remove_paragraph]

In [11]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4-turbo")

In [12]:
from langchain.agents import create_tool_calling_agent

agent = create_tool_calling_agent(llm, tools, prompt)

In [13]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [14]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [15]:
message_history = ChatMessageHistory()

In [16]:
agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    lambda session_id: message_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [17]:
agent_with_chat_history.invoke(
    {"input": "Hi, I'm Nacho. I'm a writer and I need help with my article."},
    config={"configurable": {"session_id": "<foo>"}},
)



> Entering new AgentExecutor chain...
Hello Nacho! I'd be happy to help with your article. What specific assistance do you need? Are you looking to develop ideas, refine the text, or something else? Let me know more about your article and how I can assist you!

> Finished chain.


{'input': "Hi, I'm Nacho. I'm a writer and I need help with my article.",
 'chat_history': [],
 'output': "Hello Nacho! I'd be happy to help with your article. What specific assistance do you need? Are you looking to develop ideas, refine the text, or something else? Let me know more about your article and how I can assist you!"}

In [20]:
input = """I want to include that Suno AI was developed by Anthropic, a group of ex-OpenAI researchers and the same ones who developed Claude 3, a powerful model that copete directly with GPT-4 that i will talk about later in this article. Put that where i write about Suno AI.
"""
agent_with_chat_history.invoke(
    {"input": input},
    config={"configurable": {"session_id": "<foo>"}},
)



> Entering new AgentExecutor chain...

Invoking: `modify_paragraph` with `{'index': 2, 'modified_paragraph': "Suno AI, which burst onto the scene in December 2023, quickly struck a chord in the world of music technology. Developed by Anthropic, a creative team in Cambridge, MA comprising ex-OpenAI researchers who also developed Claude 3, a powerful model that competes directly with GPT-4, Suno AI is not just a music generator; it's a revolutionary force transforming how we interact with music. At the heart of Suno are its two main AI models, Bark and Chirp. Bark focuses on delivering soul-stirring vocal melodies, while Chirp handles the intricate instrumental backdrops, allowing users to transform simple text prompts into rich musical compositions. This AI marvel makes creating radio-quality music accessible to everyone, regardless of their musical expertise, through its innovative platform."}`


Paragraph modifiedI've updated the section about Suno AI to include that it was develope

{'input': 'I want to include that Suno AI was developed by Anthropic, a group of ex-OpenAI researchers and the same ones who developed Claude 3, a powerful model that copete directly with GPT-4 that i will talk about later in this article. Put that where i write about Suno AI.\n',
 'chat_history': [HumanMessage(content="Hi, I'm Nacho. I'm a writer and I need help with my article."),
  AIMessage(content="Hello Nacho! I'd be happy to help with your article. What specific assistance do you need? Are you looking to develop ideas, refine the text, or something else? Let me know more about your article and how I can assist you!"),
  HumanMessage(content="\nI'm writting about AI music, can you help me? I already have something written. Check it out please.\n"),
  AIMessage(content="Thanks for sharing your article, Nacho! Here's what you've written so far:\n\n---\n\n**Music Generation**\n\nI had another interesting topic for this week, revolving around World Models in LLMs, but the AI scene wa